In [10]:
from picamera2 import Picamera2
import hailo


def stream_from_camera(camera_num=0):
    camera = Picamera2(camera_num)
    return camera

def release_camera(camera):
    Picamera2.release(camera)
    Picamera2.close(camera)

try:
    camera_stream_0 = stream_from_camera(0)
except RuntimeError:
    # Camera in Acquired state trying acquire() requiring state Available
    release_camera(0)


pipeline = hailo.Pipeline()



Camera __init__ sequence did not complete.
[11:04:07.861750948] [2653] ERROR Camera camera.cpp:675 Camera in Acquired state trying acquire() requiring state Available


AttributeError: type object 'Picamera2' has no attribute 'release'

In [15]:
%pip install PyAudio py-picotts adafruit-circuitpython-servokit

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Note: you may need to restart the kernel to use updated packages.


In [9]:
import time
import wave
import io
import subprocess
import random
from picotts import PicoTTS
import pyaudio


def initialize_audio():
    """
    Get the system ready for audio output
    """
    
    subprocess.run(['amixer', '-D', 'pulse', 'sset', 'Master', '100%'])
    subprocess.run(['amixer', '-D', 'pulse', 'sset', 'Master', 'bass', '10%'])
    subprocess.run(['amixer', '-D', 'pulse', 'sset', 'Master', 'treble', '90%'])

    return "Audio Initialized"


# PicoTTS voices are ['de-DE', 'en-GB', 'en-US', 'es-ES', 'fr-FR', 'it-IT']
picotts = PicoTTS()
picotts.voice = 'en-US'


def empire_demo():
# Open the assets/texts/empire_strikes_back.txt file
    with open('assets/texts/empire_strikes_back.txt', 'r') as f:
        text = f.read()

    lines = text.split('\n')

    for line in lines:
        wavs = picotts.synth_wav(line)
        p = pyaudio.PyAudio()
        stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, output=True)
        stream.write(wavs)
        stream.stop_stream()
        stream.close()
        p.terminate()
        time.sleep(1)

    return "Empire Demo Complete"


def creepy_quotes():
    # Open the assets/texts/cry_for_help.txt file
    with open('assets/texts/creepy_quotes.txt', 'r') as f:
        text = f.read()

    lines = text.split('\n')

    for line in lines:
        wavs = picotts.synth_wav(line)
        p = pyaudio.PyAudio()
        stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, output=True)
        stream.write(wavs)
        stream.stop_stream()
        stream.close()
        p.terminate()
        time.sleep(1)

    return "Cry for Help Demo Complete"

creepy_quotes()

KeyboardInterrupt: 

In [59]:
import time
import random
from adafruit_servokit import ServoKit


# Hat is addressed at 0x42
kit = ServoKit(channels=16, address=0x42)

tilt_servo = kit.servo[0]
pan_servo = kit.servo[1]

tilt_servo.set_pulse_width_range(900, 2600)
pan_servo.set_pulse_width_range(2850, 890)


# Dicts that adjust the servo angles to the correct values
tilt_dict = {
    'up': 90,
    'forward': 180,
    'backward': 0
}


pan_dict = {
    'left': 0,
    'center': 90,
    'right': 180
}

def tilt_full_direction(direction):
    tilt_servo.angle = tilt_dict[direction]
    return f"Tilted {direction}"

def pan_full_direction(direction):
    pan_servo.angle = pan_dict[direction]
    return f"Panned {direction}"

def tilt(direction):
    # Get the current angle of the tilt servo
    current_angle = tilt_servo.angle
    # go towards the desired angle by 10 degrees
    if direction == 'up':
        tilt_servo.angle = current_angle + 10
    elif direction == 'down':
        tilt_servo.angle = current_angle - 10
    return f"Tilted {direction}"

def pan(direction):
    # Get the current angle of the pan servo
    current_angle = pan_servo.angle
    # go towards the desired angle by 10 degrees
    if direction == 'left':
        pan_servo.angle = current_angle - 10
    elif direction == 'right':
        pan_servo.angle = current_angle + 10
    return f"Panned {direction}"

def tilt_to(angle):
    tilt_servo.angle = angle
    return f"Tilted to {angle}"

def pan_to(angle):
    pan_servo.angle = angle
    return f"Panned to {angle}"

def reset_servos():
    tilt_servo.angle = tilt_dict['forward']
    pan_servo.angle = pan_dict['center']
    return "Servos Reset"

def move_servos():
    tilt_servo.angle = 0
    pan_servo.angle = 0
    time.sleep(1)
    tilt_servo.angle = 180
    pan_servo.angle = 180
    time.sleep(1)
    tilt_servo.angle = 90
    pan_servo.angle = 90
    return "Servos Moved"

def move_servos_random():
    tilt_servo.angle = random.randint(0, 180)
    pan_servo.angle = random.randint(0, 180)
    return "Servos Moved Randomly"

print("Servos Initialized")
reset_servos()
time.sleep(1)
print(move_servos())
time.sleep(1)
print(move_servos_random())
time.sleep(1)
print(tilt_full_direction('up'))
time.sleep(1)
print(tilt_full_direction('forward'))
time.sleep(1)
print(tilt_full_direction('backward'))
time.sleep(1)
print(pan_full_direction('left'))
time.sleep(1)
print(pan_full_direction('center'))
time.sleep(1)
print(pan_full_direction('right'))
time.sleep(1)
print(tilt('up'))
time.sleep(1)
print(tilt('down'))
time.sleep(1)
print(pan('left'))
time.sleep(1)
print(pan('right'))
time.sleep(1)
print(tilt_to(90))
time.sleep(1)
print(pan_to(90))
time.sleep(1)
print(reset_servos())
print("Servos Demo Complete")



Servos Initialized
Servos Moved
Servos Moved Randomly
Tilted up
Tilted forward
Tilted backward
Panned left
Panned center
Panned right


KeyboardInterrupt: 

In [4]:
%pip install retry

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Note: you may need to restart the kernel to use updated packages.


In [25]:
import re
import time
import requests
import pyaudio
from retry import retry


COQUI_URL = "http://[::1]:5002"
COQUI_ENDPOINT = "/api/tts?text="


@retry(delay=2, backoff=3, max_delay=10)
def coqui_tts(text):
    """
    Send the text to the Coqui TTS server and return the audio
    """
    response = requests.get(COQUI_URL + COQUI_ENDPOINT + text)
    audio = response.content
    return audio


@retry(delay=2, backoff=3, max_delay=10)
def play_audio(audio):
    """
    Play the audio using PyAudio
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, output=True)
    stream.write(audio)
    stream.stop_stream()
    stream.close()
    p.terminate()
    return "Audio Played"


def coqui_demo():
    """
    Run the Coqui TTS demo
    """
    text = "Hello, my name is Coqui. I am a text to speech engine."
    audio = coqui_tts(text)
    play_audio(audio)
    return "Coqui Demo Complete"


def empire_demo():
# Open the assets/texts/empire_strikes_back.txt file
    with open('assets/texts/empire_strikes_back.txt', 'r') as f:
        text = f.read()

    lines = text.split('\n')
    
    # Remove any empty lines
    lines = [line for line in lines if line]

    # Remove any lines that are just whitespace
    lines = [line for line in lines if not re.match(r'^\s*$', line)]

    # Remove any consecutive punctuation characters, replacing them with a single character.
    lines = [line.replace('..', '.').replace('!!', '!').replace('??', '?') for line in lines]

    # Remove any tabs, newlines, or extra spaces
    lines = [line.replace('\t', ' ').replace('\n', ' ').replace('\r', ' ') for line in lines]

    lines = [line.strip() for line in lines]

    # Join all lines into a single string
    text = ' '.join(lines)

    while '  ' in text:
        text = text.replace('  ', ' ')

    # Break the lines into sentences
    sentences = text.split('.')

    for sentence in sentences:
        print(sentence)
        audio = coqui_tts(sentence)
        play_audio(audio)


def text_to_speech(text):
    """
    Convert text to speech using Coqui TTS
    """
    audio = coqui_tts(text)
    play_audio(audio)
    return "Text to Speech Complete"


if __name__ == "__main__":
    #coqui_demo()
    empire_demo()
    #text_to_speech("Hello, my name is Coqui. I am a text to speech engine.")


EXT
 GALAXY - PLANET HOTH A Star Destroyer moves through space, releasing Imperial probe robots from its underside
 One of these probes zooms toward the planet Hoth and lands on its ice- covered surface
 An explosion marks the point of impact
 EXT
 HOTH - METEORITE CRATER - SNOW PLAIN - DAY A weird mechanical sound rises above the whining of the wind
 A strange probe robot, with several extended sensors, emerges from the smoke-shrouded crater
 The ominous mechanical probe floats across the snow plain and disappears into the distance
 EXT
 PLAIN OF HOTH - DAY A small figure gallops across the windswept ice slope
 The bundled rider is mounted on a large gray snow lizard, a Tauntaun
 Curving plumes of snow rise from beneath the speeding paws of the two-legged beast
 The rider gallops up a slope and reins his lizard to a stop
 Pulling off his protective goggles, Luke Skywalker notices something in the sky
 He takes a pair of electrobinoculars from his utility belt and through them sees smo

KeyboardInterrupt: 

NameError: name 'paragraphs' is not defined